# Logistic Ensemble
: 여러개의 로지스틱 회귀 모델을 조합하여 더 나은 예측 성능

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Create dataset, dataloader

In [2]:
dataset = FashionMNIST(root='./data',
                       train=True,
                       transform=ToTensor(),
                       download=False)
train_set, val_set = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_set, batch_size=100, shuffle=True)
test_loader = DataLoader(val_set, batch_size=100, shuffle=False)

## Define Model

In [3]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self,x):
        out = self.linear(x)

        return out

## Set Hyperparameter

In [4]:
input_size = 28*28
num_classes = 10
num_epoches = 19
lr = 0.001
num_models = 5  #앙상블에 사용할 모델 개수

## Set model

In [5]:
models = [LogisticRegression(input_size, num_classes) for _ in range(num_models)]
print(models)

#loss, opt
criterion = nn.CrossEntropyLoss()
optimizers = [optim.SGD(model.parameters(), lr=lr) for model in models]

[LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)]


## Train

In [7]:
for epoch in range(num_epoches):
    for i, (images, labels) in enumerate(train_loader):
        # load data
        # cuda를 사용할거면 .to(device)
        images = images.reshape(-1, 28*28)
        labels = labels

        # forward,  loss
        for j in range(num_models):
            outputs = models[j](images)
            loss = criterion(outputs, labels)

            #backward, optim
            optimizers[j].zero_grad()
            loss.backward()
            optimizers[j].step()
    
    #검증코드
    with torch.no_grad():
        total, correct = 0,0
        for images, labels in test_loader:
            images = images.reshape(-1, input_size)
            labels = labels
            
            #앙상블 모델의 예측갑 더하기
            for j in range(num_models):
                outputs +=models[j](images)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels ).sum().item()

        val_acc = 100*correct/total
        print(f"Epoch [{epoch+1}/{num_epoches}], Val acc: {val_acc:.4f}")
            


Epoch [1/19], Val acc: 11.7750
Epoch [2/19], Val acc: 12.2250
Epoch [3/19], Val acc: 12.3333
Epoch [4/19], Val acc: 12.8833
Epoch [5/19], Val acc: 12.7083
Epoch [6/19], Val acc: 13.5000
Epoch [7/19], Val acc: 12.8500
Epoch [8/19], Val acc: 13.3667
Epoch [9/19], Val acc: 13.4250
Epoch [10/19], Val acc: 13.4000
Epoch [11/19], Val acc: 13.2250
Epoch [12/19], Val acc: 13.3667
Epoch [13/19], Val acc: 13.2917
Epoch [14/19], Val acc: 13.2333
Epoch [15/19], Val acc: 13.2917
Epoch [16/19], Val acc: 13.4083
Epoch [17/19], Val acc: 13.4250
Epoch [18/19], Val acc: 13.3250
Epoch [19/19], Val acc: 13.2667
